In [11]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
import pandas as pd

# Azure Form Recognizer credentials
form_recognizer_endpoint = "https://doc-intelligence-inv-pro.cognitiveservices.azure.com/"
form_recognizer_key = "6510688e806d4e44b1a0067f321574b0"
model_id = "Domestic_cust_invoice"
# Azure Blob Storage credentials
storage_connection_string = "DefaultEndpointsProtocol=https;AccountName=adlstilnvoiceinput;AccountKey=iCycvf2YkC96SU9teDiTswyf6RXxJGqE8fdznzsDw1iLiN7TzVoEzdvkSW4LFMGNr53AfzRBZxOl+AStQsaZKw==;EndpointSuffix=core.windows.net"
container_name = "test"

# Create clients
form_recognizer_client = DocumentAnalysisClient(
    endpoint=form_recognizer_endpoint,
    credential=AzureKeyCredential(form_recognizer_key)
)
blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [12]:
def analyze_document(blob_url):
    poller = form_recognizer_client.begin_analyze_document_from_url(model_id, blob_url)
    result = poller.result()
    return result

In [13]:
results = []
for blob in container_client.list_blobs():
    blob_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob.name}"
    print(f"Analyzing invoice from blob: {blob.name}")
    analysis_result = analyze_document(blob_url)
    for doc in analysis_result.documents:
        doc_fields = {field_name: field.value for field_name, field in doc.fields.items()}
        results.append(doc_fields)

Analyzing invoice from blob: 2324Y DIST 00290.pdf
Analyzing invoice from blob: BILL No B3 SCAN04042023_0000.pdf
Analyzing invoice from blob: BILL_NO._1582.pdf
Analyzing invoice from blob: BILL_NO._21.pdf
Analyzing invoice from blob: BILL_NO._3.pdf
Analyzing invoice from blob: CamScanner 05-18-2023 16.02.pdf
Analyzing invoice from blob: CamScanner 05-30-2023 15.30.pdf
Analyzing invoice from blob: GJ12BV7564 - inv 250 - 8849638.pdf
Analyzing invoice from blob: Lokmangal Invoice.pdf
Analyzing invoice from blob: Meraki Inv 00003.pdf
Analyzing invoice from blob: NL01AF4397 - inv 271 - LR 1034.pdf
Analyzing invoice from blob: PI-AG-EGJ-2223-0005_INV-22-23-42.pdf
Analyzing invoice from blob: PI-AG-MH-2021-1216_SB-20-00114.pdf
Analyzing invoice from blob: PI-AG-MH-2021-1217_SB-20-00115.pdf
Analyzing invoice from blob: PI-AG-MH-2021-1218_SB-20-00116.pdf
Analyzing invoice from blob: PI-AG-MH-2021-3034_ZHCO22021037.pdf
Analyzing invoice from blob: PI-AG-MH-2122-1208_VRS-2021-22-1029.pdf
Analyzing

In [14]:
df = pd.DataFrame(results)
df

,Vendor Invoice no.,Vendor name,Vendor Address,Vendor GST no.,Vehicle no.,Sr no.,Description of Goods (Product Name),HSN no.,Product Qty,Unit,Product Rate,Product Amount,Total Value,Total Tax Value(GST),Details of receiver / Billed to,Billing Address,Details of consignee/ Shipped to,Shipping Address
0,2324Y-DIST-00290,Jaywant Sugars Ltd.(Distillery Division),"Dhawarwadi, Tal. Karad, Dist. Satara 415109, M...",27AABCJ6812D1ZU,MH04LQ9154,1,C-Heavy Molasses Denatured Anhydrous Ethanol,29121910 29239000,40000.00,Liter,55.000,0.00 0.00,"2,596,000.00",396000.00,TATA INTERNATIONAL LIMITED,"Vios Tower, 24th Floor, New Cuffe Parade(NCP),...",NAYARA ENERGY LTD MUMBAI,"Dhrubajyoti Saharay, Plot no.72, M/s Aegis Log..."
1,23-24/B3,VJAGDISHKUMAR INDUSTRIES PRIVATE LIMITED,BBZ-N-67 ZANDA CHOWK GANDIDHAM KUTCH,24AAGCV1270L1Z8,None,1,Unregistered Rice RAW RICE SILKY SORTEX 25 % B...,None,648.500,Qtls.,"2,650.00","17,18,525.00","17,18,525.00",None,"Tata International Limited""","Office No :- 11, Ground Floor , Plot No .40 Se...",Tata International Limited,"Office No :- 11, Ground Floor , Plot No .40 Se..."
2,1582,BABA RAMDEO SILKY RICE MILL,"BAJPAI WARD,GONDIA-441601",27AAHFB5713G1ZM,MH40-CM3272,1.,Rice,1006,270.00,None,"2,745.00","7,41,150.00","7,41,150.00",0.00,TATA INTERNATIONAL LIMITED,"OPPOSITE IMAX DOME,24 FLOOR LODHA EXCELUS,NEW ...",TATA INTERNATIONAL LIMITED,"OPPOSITE IMAX DOME,24 FLOOR LODHA EXCELUS,NEW ..."
3,21,BABA RAMDEO SILKY RICE MILL,"BAJPAI WARD,GONDIA-441601",27AAHFB5713G1ZM,MH40-BL6572,1.,Rice,1006,270.00,None,"2,745.00","7,41,150.00","7,41,150.00",0.00,TATA INTERNATIONAL LIMITED,"OPPOSITE IMAX DOME,24 FLOOR , LODHA EXCELUS,NE...",TATA INTERNATIONAL LIMITED,"OPPOSITE IMAX DOME,24 FLOOR LODHA EXCELUS, NEW..."
4,3,BABA RAMDEO SILKY RICE MILL,"BAJPAI WARD,GONDIA-441601",27AAHFB5713G1ZM,MH40CD5853,1.,Rice,1006,270.00,None,"2,745.00","7,41,150.00","7,41,150.00",0.00,TATA INTERNATIONAL LIMITED,"OPPOSITE IMAX DOME,24 FLOOR LODHA EXCELUS,NEW ...",TATA INTERNATIONAL LIMITED,"OPPOSITE IMAX DOME,24 FLOOR LODHA EXCELUS,NEW ..."
5,KR23/39,KANAIYA TRADING CO,31-BHOJARAJPARA NEAR DOLAT MILL,24BRFPD7643K1,GJ14X9061,1,CHANA,None,0.000,Kg,49.00,245490.00,"2,45,490.00",None,TATA INTERNATIONAL LIMITED,OFFICE NO-11 GROUND FLOOR PLT NO 40 SECTOR NO-...,None,None
6,KR23/44,KANAIYA TRADING CO,31-BHOJARAJPARA NEAR DOLAT MILL,24BRFPD7643K1,GJ11U 9103,1,CHANA,None,0.000,Kg,49.11,523513.00,"5,23,513.00",None,TATA INTERNATIONAL LIMITED,OFFICE NO-11 GROUND FLOOR PLT NO 40 SECTOR NO-...,None,None
7,M-A000250,THE BATALA CO.OP. SUGAR MILLS LIMITED,"BATALA, P.O.BOX NO.143505 DISTRICT GURDASPUR",03AAA AT092 5E1Z3,GJ12BV7564,1,MOLASSES,17031000,399.95,None,708.00,283164.,283449,None,M/s TATA INTERNATIONAL LTD,"OFFICE NO. 11,GROUND FLOOR PLOT NO. 40, SECTOR...",M/s SHREEJI LIQUID STORAGE TERMINAL,PLOT NO 7 JAWAHAR ROAD KANDLA GANGHIDHAM GUJAR...
8,2324/LADIT/947,LOKMANGAL AGRO INDUSTRIES LTD (DISTILLERY DIVI...,"Subhash Nagar, A/p, Bibi Darphal, Tal. North S...",27AAACL7933N1Z0,MH43BG1925,1,DEN. ABSOLUTE ALCOHOL (ETHANOL) BY B-HEAVY MOL...,22072000,29000,BL,57.250,1660250.00,1959095.00,None,TATA INTERNATIONAL LTD.,"OPPOSITE IMAX DOME, 24 FLOOR, VIOS TOWER, NEW ...",MS. NAYARA ENERGY LTD,"AEGIS COASTAL TERMINAL,PLOT NO-72,AMBAPADA MAH..."
9,3,MERAKI FOODS,"NEAR ARDC GODOWN, SIMBHAL ROAD, TOHANA-125120,...",06AAHPN5398K1Z3,MH04FU6878,1,RICE 25KGS-ARIMA,1006,1000,None,2700.00,675000.00,708750.00,33750.00,M/s.TATA INTERNATIONAL LIMITED,"OPPOSITE IMAX DOME, 24TH FLR LODHA EXCELUS, NE...",PREETI CFS,"YARD II - PREETI CFS, SURVEY NO. 137/1A AT AJI..."


In [6]:
# Specify your desired Excel file name
excel_file_name = "test_Domestic_Invoice_output.xlsx"
 
# Save the DataFrame to an Excel file
df.to_excel(excel_file_name, index=False)

In [4]:
# import libraries
import os
import pandas as pd
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient

# set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
endpoint = "https://doc-intelligence-inv-pro.cognitiveservices.azure.com/"
key = "6510688e806d4e44b1a0067f321574b0"
blob_connection_string = "DefaultEndpointsProtocol=https;AccountName=adlstilnvoiceinput;AccountKey=iCycvf2YkC96SU9teDiTswyf6RXxJGqE8fdznzsDw1iLiN7TzVoEzdvkSW4LFMGNr53AfzRBZxOl+AStQsaZKw==;EndpointSuffix=core.windows.net"  # Connection string for the Azure Storage Account containing blob container
blob_container_name = "agritest"  # Name of the blob container containing the invoice files

In [17]:
invoiceUrl = "https://adlstilnvoiceinput.blob.core.windows.net/agritest/Lokmangal Invoice.pdf"

document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

poller = document_analysis_client.begin_analyze_document_from_url(
    "prebuilt-invoice", invoiceUrl
)
invoices = poller.result()

In [21]:
for idx, invoice in enumerate(invoices.documents):
    vendor_address = invoice.fields.get("VendorAddress").content


vendor_address

'Subhash Nagar, A/p, Bibi Darphal, Tal. North Solapur, Dist. Solapur-413222'